In [1]:
# import the necessary packages
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.ops as ops
from net import Net
import cv2
import random
import time

import warnings
#suppress warning
warnings.filterwarnings('ignore')

In [3]:
face_detection_net = Net()
face_detection_net.load_state_dict(torch.load("./saved_model.pth",map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
image_path = "./image_face_detection/detection_test1.jpg"
image = cv2.imread(image_path)

In [5]:
# initialize OpenCV's selective search implementation and set the
# input image
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)

In [16]:
print("[INFO] using *quality* selective search")
#ss.switchToSelectiveSearchQuality()
ss.switchToSelectiveSearchQuality()

[INFO] using *quality* selective search


In [17]:
start = time.time()
rects = ss.process()
end = time.time()

print(len(rects))

16452


In [18]:
transform = transforms.Compose([ 
    transforms.ToPILImage(),
    transforms.Grayscale(),
    transforms.ToTensor(),    
    transforms.Resize((36,36))
])

In [19]:
(H, W) = image.shape[:2]

rois = []
locs = []

for (x, y, w, h) in rects:
	# if the width or height of the region is less than 10% of the
	# image width or height, ignore it (i.e., filter out small
	# objects that are likely false-positives)
	if w / float(W) < 0.1 or h / float(H) < 0.1:
		continue
	# extract the region from the input image, convert it from BGR to
	# RGB channel ordering
	roi = image[y:y + h, x:x + w]
	roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

	# further preprocess by the ROI

	roi_tensor_gray = transform(roi)
	
	# update our list of ROIs and associated coordinates
	rois.append(roi_tensor_gray)
	locs.append((x, y, x + w, y + h))

In [20]:
print(len(rois))
print(len(locs))

4563
4563


In [21]:
stacked_tensor = torch.stack(rois, dim=0)

print(stacked_tensor.size())

output = face_detection_net(stacked_tensor)

torch.Size([4563, 1, 36, 36])


In [22]:
probs = torch.nn.functional.softmax(output, dim=1)
probs_list = probs.tolist()

labels = {'valid_probs': [],
          'boxes': []}

In [23]:
for i in range(0,len(probs_list)):
    if (probs_list[i][1] >= 0.95):
        box = locs[i]

        labels['valid_probs'].append(probs_list[i][1])
        labels['boxes'].append(box)

tensor_boxes = torch.Tensor(labels['boxes'])
tensor_probs = torch.Tensor(labels['valid_probs'])

valid_box = ops.nms(tensor_boxes, tensor_probs, iou_threshold=0.1)
        

In [24]:
print(labels['boxes'][1])

print(valid_box)

(623, 124, 736, 248)
tensor([3, 7, 6])


In [25]:
 
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 1 px 
thickness = 1

img_color = cv2.imread(image_path)
  
# Using cv2.rectangle() method 
# Draw a rectangle with blue line borders of thickness of 1 px 
for index in valid_box:
    box = labels['boxes'][index]
    (x,y,z,t) = box
    cv2.rectangle(img_color, (x,y), (z,t), color, thickness)


cv2.imshow('image', img_color)

# add wait key. window waits until user presses a key
cv2.waitKey(0)
# and finally destroy/close all open windows
cv2.destroyAllWindows()

In [8]:
# loop over the region proposals in chunks (so we can better
# visualize them)
for i in range(0, len(rects), 100):
	# clone the original image so we can draw on it
	output = image.copy()
	# loop over the current subset of region proposals
	for (x, y, w, h) in rects[i:i + 100]:
		# draw the region proposal bounding box on the image
		color = [random.randint(0, 255) for j in range(0, 3)]
		cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
	# show the output image
	cv2.imshow("Output", output)
	key = cv2.waitKey(0) & 0xFF
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

KeyboardInterrupt: 